<a href="https://colab.research.google.com/github/ohoelske/Databases/blob/main/210116_Kursaufgabe_Woche_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

210116_Kursaufgabe Woche 2